## 1. Introduction
In a certain medieval universe, there are n Kings and t trading houses. Each trading house discloses their trades on a daily basis to advertise their influence/power, while Kings have an agreement with the Emperor only disclose weekly. The Emperor, however would like to know about potential activities of his Kings in a timely manner, without raising any unnecessary question from his kings. Hence he hires you to figure out if you can back out the Kings activities through the trading houses.

# 2. Simplified Version
To start this problem let us assume that we have only 2 kings and 2 trading houses.
We assume that we can model activity by the equation

\begin{align}
 K_{i}(t) =  \sum_j \pi_{ij} T_j(t) + \epsilon(t) \,,\text{where } \vec{\pi}_i  \sim \text{Dir}(\vec{\pi}_i | \vec{\alpha})
\end{align}
that is $\vec{\pi}_{i}$ is a latent variable, following a dirichlet distribution, while K_i follows a normal distribution
\begin{equation}
K_{i} \sim \text{N}(K_i | \vec{T}^\top \vec{\pi})
\end{equation}

Note that there is an important subtility in the difference between the random variable $K(t)$ and $\pi$: $K(t)$ and hence $\epsilon_t$ are actually $n$ random variables which are sampled from the same distribution, while $\pi$ is sampled only once in the beginning and is assumed to be constant throughout the process.

In [1]:
import numpy as np
import pymc
import matplotlib
import mymodel

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

%load_ext autoreload
%autoreload 2

Lets first start with simulating some training data

In [2]:
import numpy as np
#from pymc3 import DiscreteUniform, Exponential, deterministic, Poisson, Uniform
import pymc3 as pm

# 3. Create a mock model

In [4]:
#Make Fake Model
T1 = np.array([50, 20, 30, 40, 100, 200, 30, 40, 50, 70])
T2 = np.array([10,  5, 60, 10,  30,  40, 20, 10,100, 30])
T = np.vstack([T1,T2]).T
sd = 1

def make_noise(n, sd):
    model = pm.Model()
    with model:
        mu1 = pm.Normal("mu1", mu=0, sd=sd, shape=1)
    with model:
        step = pm.NUTS()
        trace = pm.sample(n, tune=1000, init=None, step=step, njobs=1)
    values = trace.get_values(trace.varnames[0])
    return(values)

epsilon = make_noise(T.shape[0], sd)
pi1 = np.array([0.3, 0.7])
K = T.dot(pi1) + epsilon.T

100%|██████████| 1010/1010 [00:00<00:00, 2334.81it/s]


We have now prepared all the key variables for our toy problem. Let us now see if we can recover the $\epsilon$ parameters

In [30]:
values

array([[-0.66031961],
       [ 0.72115539],
       [-0.07179911],
       [-0.18041929],
       [-0.06098157],
       [ 0.40358477],
       [ 0.22526869],
       [ 0.51429888],
       [ 0.14866975],
       [ 0.89518186]])

In [23]:
print(dir(trace))
print(trace.varnames)
values = trace.get_values(trace.varnames[0])
values.tolist()


['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattr__', '__getattribute__', '__getitem__', '__hash__', '__init__', '__len__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_attrs', '_slice', '_straces', 'add_values', 'chains', 'get_sampler_stats', 'get_values', 'nchains', 'point', 'points', 'stat_names', 'varnames']
['mu1']


[[-0.8716893166071271,
  -0.4621939665389959,
  -0.12394700718056278,
  0.21267248042906062,
  -0.47769780888567204,
  -1.3106392376085694,
  -0.48492994029033953,
  0.4774603458309812,
  0.6857377884174385,
  0.7806631975336193],
 [1.0006875318377246,
  0.9323581738287361,
  -0.8124370254796196,
  -1.4003179715280343,
  0.502009809656636,
  1.0735568563791855,
  0.24158634059068018,
  0.4862394105074077,
  -0.3473233362461149,
  -0.5660178694138237],
 [-0.07443651928788764,
  -2.462352067457748,
  1.4052266901816295,
  0.2228019915746845,
  -1.6899065744998656,
  -0.6276228024873023,
  0.1293371867214361,
  0.45211054595229905,
  0.4843730345821212,
  2.0299955383336687],
 [0.04260350839653196,
  1.2875680705599655,
  -0.26789543328935794,
  -0.03049096040688465,
  2.2359769304327943,
  1.0653651340049466,
  0.37380629599138193,
  0.26195675766120163,
  -0.3901631358111677,
  -2.0395544010522637],
 [0.9768651170952035,
  -0.9672756800824569,
  0.09173473484363037,
  0.4895655942969209

In [22]:
pymc.Normal('k',mu=0,tau=.01)

array([[ 50,  20,  30,  40, 100, 200],
       [ 10,   5,  60,  10,  30,  40]])